# 🧪 Proyecto Integrador Mid 1: API → DB → Parquet con Orquestación

Objetivo: construir un pipeline reproducible que extrae datos desde una API, valida y transforma, carga a una base de datos y exporta a Parquet, orquestado con Airflow.

- Duración: 120–150 min
- Dificultad: Media
- Prerrequisitos: Airflow básico, SQL, validación de datos

## 1. Diseño del flujo

- Extract: API pública (REST) con paginación y reintentos.
- Validate: esquema con Pandera/GE y controles de calidad (nulos, rangos).
- Transform: normalización de tipos, enriquecimiento simple.
- Load: upsert a DB (PostgreSQL/SQLite de demo).
- Export: particionado por fecha a Parquet.
- Orquestación: DAG diario; retries + alertas por email/logs.

## 2. Componentes del pipeline (código reusable)

In [ ]:
import os, time
import pandas as pd
import requests
from sqlalchemy import create_engine, text
from pandera import DataFrameSchema, Column, Check

BASE_URL = os.getenv('PI1_API_URL', 'https://api.publicapis.org/entries')
DB_URI = os.getenv('PI1_DB_URI', 'sqlite+pysqlite:///:memory:')
EXPORT_DIR = os.getenv('PI1_EXPORT_DIR', 'datasets/processed/pi1/')
os.makedirs(EXPORT_DIR, exist_ok=True)

def fetch_data(url=BASE_URL, max_retries=5):
    for i in range(max_retries):
        r = requests.get(url, timeout=30)
        if r.status_code == 429:
            time.sleep(2**i)
            continue
        r.raise_for_status()
        return r.json().get('entries', [])
    raise RuntimeError('Too many retries')

def validate(df: pd.DataFrame) -> pd.DataFrame:
    schema = DataFrameSchema({
        'API': Column(str),
        'HTTPS': Column(bool),
        'Category': Column(str)
    }, coerce=True)
    return schema.validate(df)

def transform(df: pd.DataFrame) -> pd.DataFrame:
    df = df[['API','Description','Auth','HTTPS','Cors','Link','Category']].copy()
    df['ingestion_ts'] = pd.Timestamp.utcnow()
    return df

def load_db(df: pd.DataFrame, uri=DB_URI):
    engine = create_engine(uri, future=True)
    df.to_sql('apis_publicas', engine, if_exists='append', index=False)
    return len(df)

def export_parquet(df: pd.DataFrame, out_dir=EXPORT_DIR):
    out_path = os.path.join(out_dir, f
)
    df.to_parquet(out_path, index=False)
    return out_path

def run_pipeline():
    items = fetch_data()
    df = pd.DataFrame(items)
    df = validate(df)
    df = transform(df)
    n = load_db(df)
    fp = export_parquet(df)
    return {'rows': n, 'parquet': fp}

# Quick test local
res = run_pipeline()
res

## 3. Orquestación con Airflow (DAG ejemplo)

In [ ]:
dag_code = r'''
from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.utils.email import send_email

def run_pipeline_wrapper(**context):
    from pi1_module import run_pipeline
    res = run_pipeline()
    return res

default_args = {
  'owner': 'data-eng',
  'retries': 2,
  'retry_delay': timedelta(minutes=5),
}

with DAG('pi1_api_db_parquet', start_date=datetime(2025,10,1), schedule_interval='@daily', catchup=False, default_args=default_args) as dag:
    t1 = PythonOperator(task_id='run_pipeline', python_callable=run_pipeline_wrapper)
    t1
'''
print(dag_code.splitlines()[:25])

## 4. Validaciones y monitoreo

- Añadir validaciones de calidad (conteo mínimo, no duplicados).
- Alertar por email/Slack ante fallos o métricas fuera de rango.
- Publicar artefactos (Parquet) con nombres idempotentes o versionados.